In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [2]:
path = os.path.join(os.getcwd(), '..', 'DevanagariHandwrittenCharacterDataset')

train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'test')

In [3]:
def generate_data(path):
    labels = ''
    images = []
    for folder in tqdm(os.listdir(path)):
        label = folder.split('_')[-1]
        image_paths = os.listdir(os.path.join(path, folder))
        data = list(map(lambda image: plt.imread(os.path.join(path, folder, image)).reshape(1024), image_paths))
        images.extend(data)
        labels += (label + ',') * len(image_paths)
    labels = labels.split(',')
    return np.array(images), np.array(labels[:-1])

In [4]:
images, labels = generate_data(train_path)

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
images.shape

(78200, 1024)

In [7]:
lb = LabelEncoder()
encode_label = lb.fit_transform(labels)

In [8]:
one_label = tf.keras.utils.to_categorical(encode_label, num_classes= 46)

In [9]:
one_label.shape

(78200, 46)

In [10]:
images = images / 255.0

In [11]:
test_image, test_label = generate_data(test_path)
testencode_label = lb.fit_transform(test_label)
test_one_encoded_label = tf.keras.utils.to_categorical(testencode_label, num_classes = 46)
test_image = test_image / 255.0

## Time to create the neural network 
## Creating using core Tensorflow

In [12]:
num_layer = 4
layer_neuron = [1024, 1024, 1024, 46]
x = tf.placeholder(tf.float32, shape = [None, 1024], name = 'Input_Holder')
y = tf.placeholder(tf.float32, shape = [None, 46], name = 'Output_Holder')

In [15]:
def Dense(x, neurons, activation = 'relu'):
    
#     print(x.shape)
    print(neurons)
    W = tf.Variable(tf.random.normal((int(x.shape[1]), neurons)))
    B = tf.Variable(tf.random.normal([neurons, 1]))
    
    z = tf.matmul(x, W) 
    print(z.shape)
    z = tf.add(z, B)
    print(z.shape)
    if (activation == 'relu'):
        return tf.maximum(z, 0)
    elif (activation == 'sigmoid'):
        return tf.sigmoid(z)

In [16]:
layer1 = Dense(x, layer_neuron[0])
layer2 = Dense(layer1, layer_neuron[1])
layer3 = Dense(layer2, layer_neuron[2])
logits = Dense(layer3, layer_neuron[3])

1024
(?, 1024)
(1024, 1024)
1024
(1024, 1024)
(1024, 1024)
1024
(1024, 1024)
(1024, 1024)
46
(1024, 46)


ValueError: Dimensions must be equal, but are 1024 and 46 for 'Add_7' (op: 'Add') with input shapes: [1024,46], [46,1].

In [17]:
layer1 = tf.layers.dense(x, layer_neuron[0], activation= tf.nn.relu)
layer2 = tf.layers.dense(layer1, layer_neuron[1], activation= tf.nn.relu)
layer3 = tf.layers.dense(layer2, layer_neuron[2], activation= tf.nn.relu)
logits = tf.layers.dense(layer3, layer_neuron[3], activation= tf.nn.softmax)

Instructions for updating:
Use keras.layers.dense instead.


In [18]:
loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = logits))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [19]:
optimizer = tf.train.AdamOptimizer()
step = optimizer.minimize(loss_func)

In [20]:
num_epochs = 10
batch_size = 128
num_batch = images.shape[0] // batch_size
saver = tf.train.Saver()

In [ ]:
train_loss_dir = []
test_loss_dir = []

with tf.Session() as S:
    
    S.run(tf.global_variables_initializer())
    
    
    for epoch in range(num_epochs):
        
        for i in range(num_batch - 1):
            
            cur_batch = images[batch_size * i: batch_size * (i + 1)]
            y_batch = one_label[batch_size * i : batch_size * (i + 1)]
            S.run(step, feed_dict = {x: cur_batch, y: y_batch})
        
        cur_batch = images[batch_size * i:]
        y_batch = one_label[batch_size * i: ]
        S.run(step, feed_dict = {x: cur_batch, y: y_batch})
        
        train_loss = S.run(loss_func, feed_dict = {x: images, y: one_label})
        test_loss = S.run(loss_func, feed_dict = {x: test_image, y: test_one_encoded_label})
        train_loss_dir.append(train_loss)
        test_loss_dir.append(test_loss)
        print ('Iteration: %d Train Loss: %.4f, Test Loss:%.4f'%(epoch, train_loss, test_loss))
        
        saver.save(S, 'Devnagari_NN')

Iteration: 0 Train Loss: 3.8436, Test Loss:3.8436
Iteration: 1 Train Loss: 3.8436, Test Loss:3.8436
